In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from aging.plotting import format_plots, PlotConfig, save_factory, figure, legend, format_pizza_plots
from collections import Counter
from matplotlib.lines import Line2D
%matplotlib inline


In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import colorcet as cc
from operator import add
from tqdm.auto import tqdm
from functools import reduce
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from scipy.spatial.distance import squareform, pdist
from sklearn.metrics import silhouette_score
from aging.plotting import format_plots, figure, save_factory, PlotConfig, legend, add_identity

In [3]:
def mm_norm_col(column):
    return (column - column.min()) / (column.max() - column.min())

In [4]:
from matplotlib.colors import LinearSegmentedColormap
colors = ['#c7eae5','#008C8D']
custom_cmap = LinearSegmentedColormap.from_list("custom_purples", colors, N=256)
cmm=custom_cmap

image_ctx = {'image.cmap': 'cubehelix', 'image.interpolation': 'none'}

#from matplotlib.colors import LinearSegmentedColormap
# Define the color map
#colors = ['#dadaeb','#6a51a3']
#custom_cmap = LinearSegmentedColormap.from_list("custom_purples", colors, N=256)
#cmm=custom_cmap

In [5]:
keep_syllables = np.loadtxt('/n/groups/datta/win/longtogeny/data/ontogeny/version_11/to_keep_syllables_raw.txt', dtype=int)

df = pd.read_parquet('/n/groups/datta/win/longtogeny/data/ontogeny/version_11-1/longtogeny_v2_males_raw_usage_matrix_v00.parquet').astype(float)
df = df[keep_syllables].groupby(['age','uuid','mouse']).mean()

ages= df.index.get_level_values('age')
weeks = (ages * 7) // 7
df['binned_age'] = weeks
sample = df.groupby(['mouse','binned_age']).first().reset_index()
sample2 = df.groupby(['mouse','binned_age']).last().reset_index()
sample2['binned_age'] = sample2['binned_age']+0.5
df_male = pd.concat([sample,sample2])
df_male.rename(columns={'binned_age': 'age'}, inplace=True)

df_male.set_index(['age', 'mouse'], inplace=True)

In [ ]:
keep_syllables = np.loadtxt('/n/groups/datta/win/longtogeny/data/ontogeny/version_11/to_keep_syllables_raw.txt', dtype=int)

df = pd.read_parquet('/n/groups/datta/win/longtogeny/data/ontogeny/version_11-1/longtogeny_v2_females_raw_usage_matrix_v00.parquet').astype(float)
df = df[keep_syllables].groupby(['age','uuid','mouse']).mean()

ages= df.index.get_level_values('age')
weeks = (ages * 7) // 7
df['binned_age'] = weeks
sample = df.groupby(['mouse','binned_age']).first().reset_index()
sample2 = df.groupby(['mouse','binned_age']).last().reset_index()
sample2['binned_age'] = sample2['binned_age']+0.5
df_female = pd.concat([sample,sample2])
df_female.rename(columns={'binned_age': 'age'}, inplace=True)

df_female.set_index(['age', 'mouse'], inplace=True)

In [ ]:
format_plots()

In [ ]:
# avarage per age or session
m_df=df_male.groupby(['age','mouse']).mean()
avg_m_df = m_df.groupby(['age']).mean()

# normalize the data
m_norm = mm_norm_col(m_df)
avg_m_norm = mm_norm_col(avg_m_df)

# avarage per age or session
f_df=df_female.groupby(['age','mouse']).mean()
avg_f_df = f_df.groupby(['age']).mean()

# normalize the data
f_norm = mm_norm_col(f_df)
avg_f_norm = mm_norm_col(avg_f_df)

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.tsa.stattools import acf

# Assuming m_df is your DataFrame with MultiIndex (mouse, age) and 60 measurement columns

def calculate_acf(m_df, nlags=20):
    mice = m_df.index.get_level_values('mouse').unique()
    measurements = m_df.columns
    acf_results = {measurement: [] for measurement in measurements}
    
    for mouse in mice:
        for measurement in measurements:
            try:
                individual_data = m_df.xs(mouse, level='mouse')[measurement].sort_index()
                autocorr_values = acf(individual_data, nlags=nlags)
                acf_results[measurement].append(autocorr_values)
            except KeyError as e:
                print(f'Error processing {mouse} - {measurement}: {e}')
    
    return acf_results

acf_results = calculate_acf(m_df, nlags=20)

In [ ]:
def aggregate_acf(acf_results, nlags=20):
    aggregated_acf = {}
    
    for measurement, acf_values in acf_results.items():
        # Convert list of acf arrays to a DataFrame for easy aggregation
        acf_df = pd.DataFrame(acf_values)
        # Calculate mean autocorrelation for each lag
        mean_acf = acf_df.mean(axis=0)
        aggregated_acf[measurement] = mean_acf
    
    return aggregated_acf

aggregated_acf = aggregate_acf(acf_results, nlags=20)

In [ ]:
def plot_aggregated_acf(aggregated_acf, nlags=20):
    fig = figure(1.3, 1.3)
    #plt.figure(figsize=(12, 8))
    
    for measurement, mean_acf in aggregated_acf.items():
        plt.plot(range(nlags+1), mean_acf, label=measurement)
    
    plt.xlabel('Lag')
    plt.ylabel('Mean Autocorrelation')
    plt.title('Mean Autocorrelation for Each Syllable Across All Mice')
    plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
    plt.tight_layout()
    sns.despine()
    plt.show()
    c = PlotConfig()
    #fig.savefig(c.dana_save_path / "fig4"/ 'autocorr_decay_line_longv2_male.pdf')

plot_aggregated_acf(aggregated_acf, nlags=20)

In [ ]:
# Function to compute average ACF for each mouse
def compute_avg_acf(df, nlags=20):
    acf_values = {}
    for mouse in df.index.get_level_values('mouse').unique():
        mouse_data = df.xs(mouse, level='mouse')
        acf_matrix = np.array([acf(mouse_data[col], nlags=nlags, fft=False) for col in mouse_data.columns])
        avg_acf = acf_matrix.mean(axis=0)
        acf_values[mouse] = avg_acf
    return acf_values

# Compute average ACF for each mouse
nlags = 20
avg_acf_values = compute_avg_acf(m_df, nlags=nlags)

# Prepare data for plotting
lags = np.arange(nlags + 1)
colors = plt.cm.tab20(np.linspace(0, 1, len(avg_acf_values)))

In [ ]:
# Step 2: Create a DataFrame with mice as index and ACF values at different lags as columns
acf_df = pd.DataFrame(avg_acf_values).T
acf_df.columns = [i for i in range(nlags + 1)]

# Step 3: Convert the DataFrame to long format with columns: mouse, acf, and lag
acf_long_df = acf_df.reset_index().melt(id_vars='index', var_name='lag', value_name='acf')
acf_long_df.rename(columns={'index': 'mouse'}, inplace=True)

In [ ]:
format_plots()
fig, ax = plt.subplots()
fig.set_size_inches(1.3,1.3) 
sns.lineplot(data=acf_long_df, x='lag', y='acf', hue='mouse', palette = 'tab20')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xlabel('Lag (wks)')
plt.ylabel('Mean Autocorrelation')
sns.despine()

In [ ]:
c = PlotConfig()
fig.savefig(c.dana_save_path / "fig4"/ 'autocorr_ave_per_mouse_longv2_male.pdf')


In [ ]:
format_plots()
fig, ax = plt.subplots()
fig.set_size_inches(1.3,1.3) 
sns.lineplot(data=acf_long_df, x='lag', y='acf', estimator='mean',errorbar='sd', color='#20B2AA')
#plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xlabel('Lag (wks)')
plt.ylabel('Mean Autocorrelation')
sns.despine()

In [ ]:
c = PlotConfig()
fig.savefig(c.dana_save_path / "fig4"/ 'autocorr_ave_per_mouse_longv2_male_mean.pdf')

In [ ]:
# Function to compute average ACF for each mouse
def compute_avg_acf(df, nlags=20):
    acf_values = {}
    for mouse in df.index.get_level_values('mouse').unique():
        mouse_data = df.xs(mouse, level='mouse')
        acf_matrix = np.array([acf(mouse_data[col], nlags=nlags, fft=False) for col in mouse_data.columns])
        avg_acf = acf_matrix.mean(axis=0)
        acf_values[mouse] = avg_acf
    return acf_values

# Compute average ACF for each mouse
nlags = 20
avg_acf_values_f = compute_avg_acf(f_df, nlags=nlags)

# Prepare data for plotting
lags = np.arange(nlags + 1)
colors = plt.cm.tab20(np.linspace(0, 1, len(avg_acf_values)))

In [ ]:
# Step 2: Create a DataFrame with mice as index and ACF values at different lags as columns
acf_df = pd.DataFrame(avg_acf_values_f).T
acf_df.columns = [i for i in range(nlags + 1)]

# Step 3: Convert the DataFrame to long format with columns: mouse, acf, and lag
acf_long_df_f = acf_df.reset_index().melt(id_vars='index', var_name='lag', value_name='acf')
acf_long_df_f.rename(columns={'index': 'mouse'}, inplace=True)

In [ ]:
format_plots()
fig, ax = plt.subplots()
fig.set_size_inches(1.3,1.3) 
sns.lineplot(data=acf_long_df_f, x='lag', y='acf', hue='mouse', palette = 'tab20')
plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xlabel('Lag (wks)')
plt.ylabel('Mean Autocorrelation')
sns.despine()

In [ ]:
c = PlotConfig()
fig.savefig(c.dana_save_path / "fig4"/ 'autocorr_ave_per_mouse_longv2_female.pdf')

In [ ]:
format_plots()
fig, ax = plt.subplots()
fig.set_size_inches(1.3,1.3) 
sns.lineplot(data=acf_long_df_f, x='lag', y='acf', estimator='mean',errorbar='sd', color='#f16913')
#plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
plt.xlabel('Lag (wks)')
plt.ylabel('Mean Autocorrelation')
sns.despine()

In [ ]:
c = PlotConfig()
fig.savefig(c.dana_save_path / "fig4"/ 'autocorr_ave_per_mouse_longv2_female_mean.pdf')

In [ ]:
fig, ax = plt.subplots()
fig.set_size_inches(1.3,1.3) 
ax = sns.lineplot(data=acf_long_df, x='lag', y='acf', estimator='mean',errorbar='sd', color='#20B2AA')
#plt.legend(bbox_to_anchor=(1.05, 1), loc='upper left')
ax = sns.lineplot(data=acf_long_df_f, x='lag', y='acf', estimator='mean',errorbar='sd', color='#f16913')


plt.xlabel('Lag (wks)')
plt.ylabel('Mean Autocorrelation')
sns.despine()

In [ ]:
c = PlotConfig()
fig.savefig(c.dana_save_path / "fig4"/ 'autocorr_ave_per_mouse_longv2_fvm.pdf')